In [1]:
from ultralytics import YOLO
import tensorflow as tf
import numpy as np
import cv2
import os

In [2]:
LABELS = ["Boxing", "Punching", "Standing", "Holding"] 
current_directory = os.getcwd()
project_path= os.path.abspath(os.path.join(current_directory, "../.."))

In [4]:
LSTM_Model = tf.keras.models.load_model(f"{project_path}/model/best/model.keras")

In [ ]:
model = YOLO('yolov8n-pose.pt')

cap = cv2.VideoCapture(0)


steps = 0
lst_of_dct = {'key_1':[]}
label = " "
lst = []
predictions_lst = []
label_ = ''

while True:

    ret, image = cap.read()

    results = model.predict(source=image, conf=0.4, classes= 0, device=0, save=False)
    
    boxes = results[0].boxes
    result_keypoint = results[0].keypoints
   
    
    if len(result_keypoint[0].xy[0]) == 0:
        continue
    else:
        num = result_keypoint[0].xy[0].cpu().numpy()
        num = num.reshape((1, 34))
        
        lst_of_dct['key_1'].append(num)
        steps += 1
        
        if steps == 64:

            steps = 0
            lst=lst_of_dct['key_1']
            lst_of_dct = {'key_1':[]}
                
            lst = np.array(lst)
            lst = lst.reshape((1, 64, 34))
            
            preds = LSTM_Model.predict(lst)
            
            threshold = 0.8
            predicted_probabilities = preds[0]
            
            max_prob = np.max(predicted_probabilities)
        
            if max_prob > threshold:
                label = LABELS[np.argmax(preds)]
                predictions_lst.append(label)
            else:
                label = "NORMAL"
                predictions_lst.append(label)

        image = cv2.putText(image, label, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        image = cv2.resize(image, (1280, 720))
        cv2.imshow("Prediction", image)

        key = cv2.waitKey(1)
        if key == ord("a"):
            break


cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO('yolov8n-pose.pt')

# http://192.168.65.31:4747/video
cap = cv2.VideoCapture("http://192.168.65.31:4747/video")






steps = 0
lst_of_dct = {'key_1':[],'key_2':[]}


label = " "
lst = []
predictions_lst = []
label_ = ''

while True:
  
    # person1 = 0
    # person2 = 0
    minn = 1500
    points = {}
    i = 0

    ret, image = cap.read()


    results = model.predict(source=image, conf=0.60, classes= 0, device=0, save=False)
    
    boxes = results[0].boxes
    result_keypoint = results[0].keypoints
    image = cv2.putText(image, f"there is {len(result_keypoint)}", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    if len(result_keypoint) > 1:
        key1 = 0
        key2 = 0

        for box in boxes:

            coordinates = box.xyxy[0]

            x_c = (coordinates[0] + coordinates[2]) // 2
            y_c = (coordinates[1] + coordinates[3]) // 2
            p = (x_c, y_c)

            points[i] = [coordinates, p]

            for k in points.keys():
                if i != k:
                    dis = ((points[i][1][0] - points[k][1][0]) ** 2 +
                        (points[i][1][1] - points[k][1][1]) ** 2) ** 0.5

                    if dis < minn:
                        minn = dis
                        # person1 = (int(points[i][1][0]), int(points[i][1][1]))
                        # person2 = (int(points[k][1][0]), int(points[k][1][1]))
                        key1 = i
                        key2 = k
                    
            i += 1
        if key1 == 0 and key2 == 0:
            keys = []
        else:
            keys = [key1, key2]
      
        for z in range(len(keys)):

            
            num = result_keypoint[keys[z]].xy[0].cpu().numpy()
            num = num.reshape((1, 34))
        
            if z==0:
                lst_of_dct['key_1'].append(num)
                steps += 1
            if z==1:
                lst_of_dct['key_2'].append(num)

            if steps == 32:

                if z==0:
                    lst=lst_of_dct['key_1']
                if z==1:
                    lst=lst_of_dct['key_2']
                    steps = 0
                    lst_of_dct = {'key_1':[],'key_2':[]}

                lst = np.array(lst)
                lst = lst.reshape((1, 32, 34))
                
                preds = LSTM_Model.predict(lst)
               

                threshold = 0.94
                predicted_probabilities = preds[0]

                max_prob = np.max(predicted_probabilities)

                if max_prob > threshold:
                    
                    label = LABELS[np.argmax(preds)]
                    predictions_lst.append(label)
                else:
                    label = "NORMAL"
                    predictions_lst.append(label)
                
                lst = []
            if len(predictions_lst) == 8:
                
                if predictions_lst.count('BOXING')>=4:
                    label_ = "Violence"
                else:
                    label_ = "No Violence"
                predictions_lst=[]
               
            # else:
            #     label_="proccessing"
    else:
        label_="Only one person in image"
        predictions_lst = []

    image = cv2.putText(image, label_, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    cv2.imshow("Prediction", image)

    key = cv2.waitKey(1)
    if key == ord("a"):
        break


cv2.destroyAllWindows()